In [ ]:
import os
import json
import pandas as pd

#def extract_consult~ 함수에서 추출된 경로를 형식에 맞게 수정하는 함수
def modify_path(text_path):
    # Base directories
    base_text_dir = "D:\\user\\Downloads\\상담 음성\\Training"
    base_audio_dir = "D:\\user\\Downloads\\상담 음성\\Training"

    # Extract folder names from the paths
    parts = text_path.split('/')
    D_folder = parts[1]
    J_folder = parts[2]
    S_folder = parts[3]
    file_name = parts[4]

    # Modify the textPath
    text_folder = f"[라벨]KtelSpeech_train_{D_folder}_label_0"
    new_text_path = os.path.join(base_text_dir, text_folder, D_folder, J_folder, S_folder, file_name)

    # Modify the audioPath
    audio_folder = f"[원천]KtelSpeech_train_{D_folder}_wav_0"
    new_audio_path = os.path.join(base_audio_dir, audio_folder, D_folder, J_folder, S_folder, file_name.replace('.txt', '.wav'))
    
    return new_audio_path, new_text_path

#json파일로 남자상담원만 추출하는 함수
def extract_consultant_dialogs(json_directory_path, csv_output_path):
    # 결과를 저장할 DataFrame을 생성합니다.
    columns = ['audioPath', 'textPath']
    result_df = pd.DataFrame(columns=columns)

    # 지정된 디렉토리에서 JSON 파일들을 읽어옵니다.
    for filename in os.listdir(json_directory_path):
        if filename.endswith('.json'):
            json_file_path = os.path.join(json_directory_path, filename)
            
            # JSON 파일을 로드합니다.
            with open(json_file_path, 'r', encoding='utf-8-sig') as file:
                json_content = json.load(file)
            
            # speakers 섹션에서 상담원의 정보를 찾습니다.
            speakers = json_content['dataSet']['typeInfo']['speakers']
            consultant = None
            for speaker in speakers:
                if speaker['type'] == '상담원' and speaker['gender'] == '남':
                    consultant = speaker
                    break
            
            # 남성 상담원이 있는 경우에만 처리를 진행합니다.
            if consultant:
                consultant_id = consultant['id']
                
                # dialogs 섹션에서 상담원의 ID에 해당하는 'audioPath'와 'textPath' 정보를 추출합니다.
                dialogs = json_content['dataSet']['dialogs']
                for dialog in dialogs:
                    if dialog['speaker'] == consultant_id:
                        audio_path, text_path = modify_path(dialog['textPath'])
                        result_df = result_df.append({'audioPath': audio_path, 'textPath': text_path}, ignore_index=True)
    
    # 추출된 정보를 CSV 파일로 저장합니다.
    result_df.to_csv(csv_output_path, encoding='utf-8-sig', index=False)
    print("CSV file has been saved at:", csv_output_path)

    
#모든 폴더를 순회하며 json파일 위치를 자동으로 설정해주기
# 기본 경로
base_directory = "D:\\user\\Downloads\\상담 음성\\Training"

# 모든 서브 디렉터리를 순회합니다.
for sub_directory in os.listdir(base_directory):
    # "[라벨]KtelSpeech_train_"로 시작하는 디렉터리만 처리합니다.
    if sub_directory.startswith("[라벨]KtelSpeech_train_"):
        D_folder = sub_directory.split('_')[2]  # "D"로 시작하는 부분을 추출
        J_folder = "J" + str(int(D_folder[1:]) + 31)  # "J"로 시작하는 부분을 계산

        # S로 시작하는 모든 폴더를 순회합니다.
        for S_folder in os.listdir(os.path.join(base_directory, sub_directory, D_folder, J_folder)):
            if S_folder.startswith("S"):
                # 완전한 경로를 구성하고 함수를 호출합니다.
                full_path = os.path.join(base_directory, sub_directory, D_folder, J_folder, S_folder)
                csv_output_path = f'consultant_dialogs_{D_folder}_{J_folder}_{S_folder}.csv'
                extract_consultant_dialogs(full_path, csv_output_path)

#####ToDo                
#남자 상담원이 아니면 CSV로 저장하지 않는 것 고려하기(지금은 row 1줄만 있는채로 출력 [audioPath,textPath])
#CSV파일 하나만 추출하는 것 고려하기              